In [15]:
import os
import time
import platform
import requests
import browser_cookie3
from bs4 import BeautifulSoup
import time

def load_cookie_from_chrome():
    global jar

    cookie_path = None
    os_type = platform.system()
    if os_type == "Linux":
        # browser_cookie3 also checks beta version of google chrome's cookie file.
        cookie_path = os.path.expanduser(
            "~/.config/google-chrome/Default/Cookies")
        if os.path.exists(cookie_path) is False:
            cookie_path = os.path.expanduser(
                "~/.config/google-chrome-beta/Default/Cookies")
    elif os_type == "Darwin":
        cookie_path = os.path.expanduser(
            "~/Library/Application Support/Google/Chrome/Default/Cookies")
    elif os_type == "Windows":
        cookie_path = os.path.expandvars(
            "%LOCALAPPDATA%/Google/Chrome/User Data/Default/Cookies")
    else:  # Jython?
        print("지원하지 않는 환경입니다.")
        close()

    if os.path.exists(cookie_path) is False:
        print("기본 쿠키 파일 경로에 파일이 존재하지 않습니다. 아래 링크를 참조하여 쿠키 파일 경로를 지정해주세요.")
        close()

    jar = browser_cookie3.chrome(domain_name="foresttrip.go.kr")

check_date_list = [
    ["20210821", "20210822", "토", "일"],
    ["20210828", "20210829", "토", "일"],
    ["20210904", "20210905", "토", "일"],
    ["20210911", "20210912", "토", "일"],
    ["20210918", "20210919", "토", "일"]
]

hmpgId = "ID02030052"
keyword = "비학산자연휴양림"
indvz_url = "https://www.foresttrip.go.kr/indvz/main.do?hmpgId="+hmpgId
check_url = "https://www.foresttrip.go.kr/rep/or/sssn/fcfsRsrvtRcrfrDtlDetls.do"
header = {
    "Accept": "*/*",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "ko,ko-KR;q=0.9,en;q=0.8",
    "Host": "www.foresttrip.go.kr",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"
}
load_cookie_from_chrome()
print("예약가능여부 체크중...")
main_page_result = requests.get(indvz_url, cookies=jar)
soup = BeautifulSoup(main_page_result.text, "html5lib")
_csrf = soup.select("input[name='_csrf']")[0].attrs["value"]
try_count = 0
done = False

while done == False:
    time.sleep(5)
    try_count += 1
    print("{}번쨰 체크중...".format(try_count), end="\r")
    for check_date in check_date_list:
        timestamp = time.time()
        timestamp = str(timestamp).split(".")[0] + str(timestamp).split(".")[1][:3]
        net_funnel_url = "https://nf.foresttrip.go.kr/ts.wseq?opcode=5101&nfid=0&prefix=NetFunnel.gRtype=5101;&sid=service_1&aid=action2&js=yes&"+timestamp
        this_net_funnel_result = requests.get(net_funnel_url, cookies=jar)
        this_net_funnel_cookie = this_net_funnel_result.text.split(";")[1].split("='")[1][:-1]
#         print(net_funnel_url, this_net_funnel_cookie, check_url)
        this_cookie = browser_cookie3.create_cookie("www.foresttrip.go.kr", "/", 0, 0, "NetFunnel_ID", this_net_funnel_cookie)
        jar.set_cookie(this_cookie)
        ymd_str1 = check_date[0][2:4]+"/"+check_date[0][4:6]+"/"+check_date[0][6:8]+"("+check_date[2]+")"
        ymd_str2 = check_date[1][2:4]+"/"+check_date[1][4:6]+"/"+check_date[1][6:8]+"("+check_date[3]+")"
        this_check_param = {
            "_csrf" : _csrf,
            "hmpgId" : hmpgId,
            "srchInsttId" : hmpgId,
            "srchRsrvtBgDt" : check_date[0],
            "srchRsrvtEdDt" : check_date[1],
            "srchUseDt" : ymd_str1+"-"+ymd_str2,
            "srchSthngCnt" : "1",
            "srchStngNofpr" : "2",                
            "rsrvtWtngSctin" : "01",
            "cmdogYn" : "N",
            "bbqYn" : "N",
            "dsprsYn" : "N",
            "otsdWeterYn" : "N",
            "wifiYn" : "N",
            "snowPlaceYn" : "N",
            "gNowPage" : "1",
            "goodsClsscHouseCdArr" : "01001",
            "keyword" : keyword,
            "calPicker" : ymd_str1+"-"+ymd_str2
        }
        print(jar)
        this_check_result = requests.post(check_url, data=this_check_param, headers=header, cookies=jar)
        print(this_check_result.text)
        break
    break

예약가능여부 체크중...
<CookieJar[<Cookie PCID=16290189147406108413430 for .foresttrip.go.kr/>, <Cookie JSESSIONID=NRukpAuo_Ugz2aZYEPInebt4NK2hPSs4MGzyXhNz.node11 for www.foresttrip.go.kr/>, <Cookie NetFunnel_ID=5002:200:key=1BBA091DC179546CD94258AD31803DCFCF7CA7E81A8475C2EABC96391C9805B3DDAC3152DA0BAEAF5CDA2F95F87ACFEBAE21C3FD5A4171A1599CE6B462F857B69B4D78C6389B46F06BCC3605B15DC36644337158B85D0F3CA1EEA0458F9F92CD64662313EBC08057800CB022DD8A9B29312C302C30&nwait=0&nnext=0&tps=0&ttl=0&ip=nf.foresttrip.go.kr&port=443 for www.foresttrip.go.kr/>, <Cookie SCOUTER=z3vm6a0h88bbd0 for www.foresttrip.go.kr/>, <Cookie enterPopup2787=close for www.foresttrip.go.kr/>, <Cookie enterPopup3188=close for www.foresttrip.go.kr/>, <Cookie enterPopup3305=close for www.foresttrip.go.kr/>, <Cookie saveId=hrdkdh for www.foresttrip.go.kr/>]>


    


<!DOCTYPE html>
<html lang="ko">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta http-equiv="X-UA-Co

In [35]:
import os
import time
import platform
import requests
import browser_cookie3
from bs4 import BeautifulSoup
import time
from urllib import parse

check_date_list = [
    ["20210821", "20210822", "토", "일"],
    ["20210828", "20210829", "토", "일"],
    ["20210904", "20210905", "토", "일"],
    ["20210911", "20210912", "토", "일"],
    ["20210918", "20210919", "토", "일"]
]

hmpgId = "ID02030052"
keyword = "비학산자연휴양림"
indvz_url = "https://www.foresttrip.go.kr/indvz/main.do?hmpgId="+hmpgId
login_url = "https://www.foresttrip.go.kr/com/login"
check_url = "https://www.foresttrip.go.kr/rep/or/sssn/fcfsRsrvtPssblGoodsDetls.do"
header = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"
}

with requests.Session() as s:
    print("로그인 중...")
    main_page_result = s.get(indvz_url)
    soup = BeautifulSoup(main_page_result.text, "html5lib")
    _csrf = soup.select("input[name='_csrf']")[0].attrs["value"]
    
    login_data = {
        "_csrf": _csrf,
        "loginId": "hrdkdh",
        "loginPwd": "echoes78!",
        "saveId": "on",
        "targetUrl": "/indvz/main.do?hmpgId="+hmpgId
    }
    login_header = {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
        "Content-Type": "application/x-www-form-urlencoded",
         "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"
    }
    login_result = s.post(login_url, data=login_data, headers=header)
    print("예약가능여부 체크중...")
    try_count = 0
    done = False

    while done == False:
        time.sleep(2)
        try_count += 1
        print("{}번쨰 체크중...".format(try_count), end="\r")
        for check_date in check_date_list:
            timestamp = time.time()
            timestamp = str(timestamp).split(".")[0] + str(timestamp).split(".")[1][:3]
            net_funnel_url = "https://nf.foresttrip.go.kr/ts.wseq?opcode=5101&nfid=0&prefix=NetFunnel.gRtype=5101;&sid=service_1&aid=action2&js=yes&"+timestamp
            this_net_funnel_result = s.get(net_funnel_url)
            this_net_funnel_cookie = this_net_funnel_result.text.split(";")[1].split("='")[1][:-1]
#             this_net_funnel_cookie = parse.quote(this_net_funnel_cookie)
            print(net_funnel_url, this_net_funnel_cookie)
            this_cookie = browser_cookie3.create_cookie("www.foresttrip.go.kr", "/", 0, 0, "NetFunnel_ID", this_net_funnel_cookie)
            s.cookies.set_cookie(this_cookie)
            this_cookie = browser_cookie3.create_cookie(".foresttrip.go.kr", "/", 0, 0, "PCID", "16290189147406108413430")
            s.cookies.set_cookie(this_cookie)
            ymd_str1 = check_date[0][2:4]+"/"+check_date[0][4:6]+"/"+check_date[0][6:8]+"("+check_date[2]+")"
            ymd_str2 = check_date[1][2:4]+"/"+check_date[1][4:6]+"/"+check_date[1][6:8]+"("+check_date[3]+")"
            this_check_param = {
                "_csrf" : _csrf,
                "hmpgId" : hmpgId,
                "srchInsttId" : hmpgId,
                "srchRsrvtBgDt" : check_date[0],
                "srchRsrvtEdDt" : check_date[1],
                "srchUseDt" : ymd_str1+"-"+ymd_str2,
                "srchSthngCnt" : "1",
                "srchStngNofpr" : "2",                
                "rsrvtWtngSctin" : "01",
                "cmdogYn" : "N",
                "bbqYn" : "N",
                "dsprsYn" : "N",
                "otsdWeterYn" : "N",
                "wifiYn" : "N",
                "snowPlaceYn" : "N",
                "gNowPage" : "1",
                "goodsClsscHouseCdArr" : "01001",
                "keyword" : keyword,
                "calPicker" : ymd_str1+"-"+ymd_str2
            }
            print(this_check_param, header)
            this_check_result = s.get(check_url, data=this_check_param, headers=header)
            print(this_check_result.text)
            break
        break

로그인 중...
예약가능여부 체크중...
https://nf.foresttrip.go.kr/ts.wseq?opcode=5101&nfid=0&prefix=NetFunnel.gRtype=5101;&sid=service_1&aid=action2&js=yes&1629511919431 5002:200:key=1762A56C0E0BFA7828491F1A8C4BCA3E99F91AF4C0E3BB5F972DDAA93846A74DE2D6DE8587515DDBED4FF03DE3F4B487EA02DD75C07C9B6D1978DEE7416DCF3E9B4D78C6389B46F06BCC3605B15DC366388CDE48B971CF6CB35855A21CD111961BE917B98E84BF23BA876D949BB6F7EE312C302C30&nwait=0&nnext=0&tps=0&ttl=0&ip=nf.foresttrip.go.kr&port=443
{'_csrf': '6b502f98-ef2f-4f8b-a3f4-47a722270513', 'hmpgId': 'ID02030052', 'srchInsttId': 'ID02030052', 'srchRsrvtBgDt': '20210821', 'srchRsrvtEdDt': '20210822', 'srchUseDt': '21/08/21(토)-21/08/22(일)', 'srchSthngCnt': '1', 'srchStngNofpr': '2', 'rsrvtWtngSctin': '01', 'cmdogYn': 'N', 'bbqYn': 'N', 'dsprsYn': 'N', 'otsdWeterYn': 'N', 'wifiYn': 'N', 'snowPlaceYn': 'N', 'gNowPage': '1', 'goodsClsscHouseCdArr': '01001', 'keyword': '비학산자연휴양림', 'calPicker': '21/08/21(토)-21/08/22(일)'} {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 

In [36]:
s.cookies

<RequestsCookieJar[Cookie(version=0, name='PCID', value='16290189147406108413430', port=None, port_specified=False, domain='.foresttrip.go.kr', domain_specified=True, domain_initial_dot=True, path='/', path_specified=True, secure=0, expires=0, discard=False, comment=None, comment_url=None, rest={}, rfc2109=False), Cookie(version=0, name='JSESSIONID', value='-CuMjiibU5uUXg1aLFlgf8x4k8JK_C9clKBLQ0ZC.node61', port=None, port_specified=False, domain='www.foresttrip.go.kr', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=True, expires=None, discard=True, comment=None, comment_url=None, rest={'HttpOnly': None, 'SameSite': 'None'}, rfc2109=False), Cookie(version=0, name='NetFunnel_ID', value='5002:200:key=1762A56C0E0BFA7828491F1A8C4BCA3E99F91AF4C0E3BB5F972DDAA93846A74DE2D6DE8587515DDBED4FF03DE3F4B487EA02DD75C07C9B6D1978DEE7416DCF3E9B4D78C6389B46F06BCC3605B15DC366388CDE48B971CF6CB35855A21CD111961BE917B98E84BF23BA876D949BB6F7EE312C302C30&nwait=0&nnext=0&t